In [1]:
import piano_reduction.tools as pr
import piano_reduction.compute_features as cf
from piano_reduction.classes import ScoreData, Reduction
from sklearn import metrics
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
reductions = {}
for i, h in tqdm([(a, b) for a in [2, 6, 8] for b in [100, 150, 200, 250, 300]]):
    try:
        reductions['GRU 1 %d using cosi_%d' % (h, i)] = Reduction.load('models/20190318h/GRU_1 (%d) %d.pkl' % (h, i))
    except:
        pass
    try:
        reductions['GRU 2 %d using cosi_%d' % (h, i)] = Reduction.load('models/20190318h/GRU_2 (%d) %d.pkl' % (h, i))
    except:
        pass
    #print(reductions)

100%|██████████| 15/15 [02:07<00:00,  8.49s/it]


In [73]:
def check_data(name, reds):
    _, new_data = ScoreData.load(name).split_in(0.8, 0.9)
    
    features = ['active_rhythm', 'bass_line', 'entrance_effect', 'highest', 'in_chord', 'lowest', 'occurrence', 
       'onset_after_rest', 'rhythm_variety', 'strong_beats', 'sustained_rhythm', 'vertical_doubling', 'duration_length', 'offset']
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import MultinomialNB
    
    for i in [2, 6, 8]:
        training_data, _ = ScoreData.load('score_data/cosi_%d.pkl' % i).split_in(0.8, 0.9)
        training_data = training_data.df[training_data.df['x_train'] == 1].copy()

        x_train = training_data[features]
        y_train = training_data['y_train']
        model = LogisticRegression().fit(x_train, y_train)
        testing_data = new_data.df[new_data.df['x_train'] == 1].copy()
        testing_data['y_pred'] = model.predict(testing_data[features])
        new_data.df['y_pred_LR using cosi_%d' % i] = testing_data['y_pred']
        new_data.df['y_pred_LR using cosi_%d' % i] = new_data.df['y_pred_LR using cosi_%d' % i].fillna(0).astype(int)


        x_train = training_data[features]
        y_train = training_data['y_train']
        model = MultinomialNB().fit(x_train, y_train)
        testing_data = new_data.df[new_data.df['x_train'] == 1].copy()
        testing_data['y_pred'] = model.predict(testing_data[features])
        new_data.df['y_pred_NB using cosi_%d' % i] = testing_data['y_pred']
        new_data.df['y_pred_NB using cosi_%d' % i] = new_data.df['y_pred_NB using cosi_%d' % i].fillna(0).astype(int)

    new_data.df['y_pred_Keep All'] = new_data.df['x_train']
    new_data.df.columns
    
    for i in reds:
        if 'GRU' in i:
            new_data = reductions[i].predict(new_data, threshold=0.1, name='y_pred_%s' % i)
        else:
            new_data = reductions[i].predict_2(new_data, threshold=0.1, name='y_pred_%s' % i)
    return new_data

def check_data_2(name, reds, threshold=0.1):
    _, new_data = ScoreData.load(name).split_in(0.8, 0.9)
    for i in reds:
        if 'GRU' in i:
            new_data = reductions[i].predict(new_data, threshold=threshold, name='y_pred_%s' % i)
        else:
            new_data = reductions[i].predict_2(new_data, threshold=threshold, name='y_pred_%s' % i)
    return new_data

In [84]:
print('%25s\t%20s\t%20s\t%20s' % ('', 'F1 score', 'Accuracy', 'Jaccard Similarity'))

for threshold in [i * 0.05 for i in range(2, 12)]:
    print(threshold)
    name = 'score_data/cosi_8.pkl'
    reds = [i for i in sorted(list(reductions), key=lambda x:(x.split()[1], int(x.split()[2]))) if name[-5] == i[-1]]
    all_data = check_data_2(name, reds, threshold)
    for i in list(reds):
        if name[-5] != i[-1]:
            continue
        new_data = all_data.copy()
        new_data.df = new_data.df[(new_data.df['x_train'] == 1) | (new_data.df['y_train'] == 1) | (new_data.df['y_pred_%s' % i] == 1)]
        b1 = new_data.to_binary(col='y_train')
        b2 = new_data.to_binary(col='y_pred_%s' % i)
        scores = [metrics.f1_score(new_data.df['y_train'], new_data.df['y_pred_%s' % i]),
                  metrics.accuracy_score(new_data.df['y_train'], new_data.df['y_pred_%s' % i]),
                  pr.jaccard_similarity(b1, b2)
                 ]
        #if 'GRU' in i:
        #    i = '%s using cosi_%c' % (i[2:], i[0])
        #elif 'Attempt 2 both ways' in i:
        #    i = '2 GRU using cosi_%c' % i[0]
        print('%25s\t%20.9f\t%20.9f\t%20.9f' % tuple([i] + scores))
    print()

                         	            F1 score	            Accuracy	  Jaccard Similarity
0.1
   GRU 1 100 using cosi_8	         0.666666667	         0.531969309	         0.500000000
   GRU 1 150 using cosi_8	         0.720156556	         0.608219178	         0.562691131
   GRU 1 200 using cosi_8	         0.680080483	         0.565573770	         0.515243902
   GRU 1 250 using cosi_8	         0.742514970	         0.647540984	         0.590476190
   GRU 2 100 using cosi_8	         0.674329502	         0.540540541	         0.508670520
   GRU 2 150 using cosi_8	         0.688212928	         0.556756757	         0.524637681
   GRU 2 200 using cosi_8	         0.662721893	         0.540322581	         0.495575221
   GRU 2 250 using cosi_8	         0.726530612	         0.630853994	         0.570512821

0.15000000000000002
   GRU 1 100 using cosi_8	         0.712598425	         0.608579088	         0.553516820
   GRU 1 150 using cosi_8	         0.729838710	         0.632876712	         0.574603

In [12]:
print('%25s\t%20s\t%20s\t%20s' % ('', 'F1 score', 'Accuracy', 'Jaccard Similarity'))

for name in ['score_data/cosi_2.pkl', 'score_data/cosi_6.pkl', 'score_data/cosi_8.pkl']:
    reds = [i for i in sorted(list(reductions), key=lambda x:(x.split()[1], int(x.split()[2]))) if name[-5] == i[-1]]
    all_data = check_data(name, reds)
    for i in list(reds) + ['LR using cosi_2', 'LR using cosi_6', 'LR using cosi_8', 
                           'NB using cosi_2', 'NB using cosi_6', 'NB using cosi_8', 'Keep All']:
        if name[-5] != i[-1]:
            continue
        new_data = all_data.copy()
        new_data.df = new_data.df[(new_data.df['x_train'] == 1) | (new_data.df['y_train'] == 1) | (new_data.df['y_pred_%s' % i] == 1)]
        b1 = new_data.to_binary(col='y_train')
        b2 = new_data.to_binary(col='y_pred_%s' % i)
        scores = [metrics.f1_score(new_data.df['y_train'], new_data.df['y_pred_%s' % i]),
                  metrics.accuracy_score(new_data.df['y_train'], new_data.df['y_pred_%s' % i]),
                  pr.jaccard_similarity(b1, b2)
                 ]
        #if 'GRU' in i:
        #    i = '%s using cosi_%c' % (i[2:], i[0])
        #elif 'Attempt 2 both ways' in i:
        #    i = '2 GRU using cosi_%c' % i[0]
        print('%25s\t%20.9f\t%20.9f\t%20.9f' % tuple([i] + scores))
    print()

                         	            F1 score	            Accuracy	  Jaccard Similarity
   GRU 1 100 using cosi_2	         0.744360902	         0.616901408	         0.592814371
   GRU 1 150 using cosi_2	         0.770093458	         0.648571429	         0.626139818
   GRU 1 200 using cosi_2	         0.746212121	         0.620396601	         0.595166163
   GRU 1 250 using cosi_2	         0.813008130	         0.737142857	         0.684931507
   GRU 1 300 using cosi_2	         0.782608696	         0.687500000	         0.642857143
   GRU 2 100 using cosi_2	         0.794676806	         0.691428571	         0.659305994
   GRU 2 150 using cosi_2	         0.751381215	         0.615384615	         0.601769912
   GRU 2 200 using cosi_2	         0.784615385	         0.682719547	         0.645569620
   GRU 2 250 using cosi_2	         0.808853119	         0.733146067	         0.679054054
   GRU 2 300 using cosi_2	         0.828571429	         0.760000000	         0.707317073
          LR using co

In [95]:
#data = ScoreData.load('score_data/3.pkl')
data = ScoreData(pr.load_df('demo', 2))
new_data = data.copy()

for i in reductions:
    if '250' not in i or 'cosi_8' not in i:
        continue
    new_data = reductions[i].predict(new_data, threshold=0.3, name='y_pred_%s' % i)
    
output_names = []
count = 0
for i in reductions:
    if '250' not in i or 'cosi_8' not in i:
        continue
    count += 1
    new_data.df['Reduction %d' % count] = new_data.df['y_pred_%s' % i]
    output_names.append('Reduction %d' % count)
new_data.show_score(['x_train'] + output_names)

In [22]:
data = ScoreData.load('score_data/cosi_8.pkl')
data.show_score(['x_train', 'y_train'])

In [69]:
new_data.show_score(['x_train'] + output_names)

In [82]:
[i for i in sys.modules if 'piano' in i]

['piano_reduction.tools',
 'piano_reduction.compute_features',
 'piano_reduction.features',
 'piano_reduction.classes.reduction',
 'piano_reduction',
 'piano_reduction.classes',
 'piano_reduction.classes.score_data']

In [81]:
from piano_reduction.classes import ScoreData